# Asistente turístico de Tenerife (RAG + Tools)

Este cuaderno implementa un asistente con:
- **RAG** sobre `TENERIFE.pdf` (búsqueda semántica con ChromaDB y MiniLM).
- **Diálogo multiturno** con memoria corta.
- **Function Calling**:
  - `get_weather(fecha, lugar)` (pronóstico *simulado*).
  - `get_sunlight(fecha, lugar)` (amanecer/atardecer reales con Astral).
- **Parámetros del LLM** visibles.
- **Validación con Pydantic** (bonus) y **métricas de evaluación** (bonus).

In [35]:
from __future__ import annotations

import os
import json
import re
from pathlib import Path
from datetime import date, datetime, timezone

from dotenv import load_dotenv
from loguru import logger

# Rutas del proyecto 
ROOT = Path.cwd().parent if (Path.cwd().name == "notebooks") else Path.cwd()
DATA_DIR = ROOT / "data"
DB_DIR = ROOT / "chroma_db"
LOG_DIR = ROOT / "logs"
PDF_PATH = DATA_DIR / "TENERIFE.pdf"

DB_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

# Logging a fichero + consola
logger.remove()  # limpia handlers previos
logger.add(LOG_DIR / "run.log", level="INFO", rotation="1 MB", enqueue=True)
logger.add(lambda msg: print(msg, end=""))

# Variables de entorno (.env)
load_dotenv(ROOT / ".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY no configurada. Añádela en .env")

# Utilidad para marcas de tiempo coherentes en logs
def now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()


In [36]:
# SDK oficial OpenAI
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Parámetros del modelo (visibles y configurables)
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TEMPERATURE = float(os.getenv("OPENAI_TEMPERATURE", "0.20"))
TOP_P = float(os.getenv("OPENAI_TOP_P", "1.0"))
MAX_TOKENS = int(os.getenv("OPENAI_MAX_TOKENS", "600"))

print(
    f"[LLM] model={MODEL} | temperature={TEMPERATURE} | top_p={TOP_P} | max_tokens={MAX_TOKENS}"
)

[LLM] model=gpt-4o-mini | temperature=0.2 | top_p=1.0 | max_tokens=600


## Ingesta e indexación (RAG)

- Lee `TENERIFE.pdf`.
- Divide en *chunks* por tokens.
- Genera embeddings con `all-MiniLM-L6-v2`.
- Persiste en **ChromaDB**.

In [37]:
from typing import List, Tuple
import tiktoken
from pypdf import PdfReader
import chromadb
from chromadb.utils import embedding_functions

# Tokenizador y chunking
_enc = tiktoken.get_encoding("cl100k_base")

def chunk_by_tokens(text: str, chunk_size: int = 450, overlap: int = 80) -> List[str]:
    """Divide texto por tokens preservando solape."""
    ids = _enc.encode(text)
    chunks = []
    start = 0
    while start < len(ids):
        end = min(start + chunk_size, len(ids))
        piece = _enc.decode(ids[start:end])
        chunks.append(piece)
        if end == len(ids): break
        start = end - overlap
        if start < 0: start = 0
    return chunks

# Lectura PDF a (texto, página)
def read_pdf_texts(pdf_path: Path) -> List[Tuple[str, int]]:
    reader = PdfReader(str(pdf_path))
    out = []
    for i, page in enumerate(reader.pages, start=1):
        text = page.extract_text() or ""
        if text.strip():
            out.append((text, i))
    return out

# Construcción de índice vectorial persistente 
def build_or_refresh_index() -> chromadb.Collection:
    logger.info(f"[{now_iso()}] Ingesta PDF en {PDF_PATH}")
    texts_pages = read_pdf_texts(PDF_PATH)

    ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2"
    )
    client_chroma = chromadb.PersistentClient(path=str(DB_DIR))
    coll = client_chroma.get_or_create_collection(
        name="tenerife", embedding_function=ef
    )
    # Limpiamos colección para una ingesta determinista
    try:
        coll.delete(where={})
    except Exception:
        pass

    docs, metas, ids = [], [], []
    serial = 0
    for text, page in texts_pages:
        for j, chunk in enumerate(chunk_by_tokens(text)):
            docs.append(chunk)
            metas.append({"page": page, "chunk_id": j})
            ids.append(f"d{serial:06d}")
            serial += 1

    logger.info(f"[{now_iso()}] Insertando {len(docs)} chunks en Chroma…")
    coll.add(documents=docs, metadatas=metas, ids=ids)
    return coll

collection = build_or_refresh_index()
logger.info(f"[{now_iso()}] Colección lista con {collection.count()} vectores.")

2025-08-31 13:07:42.932 | INFO     | __main__:build_or_refresh_index:36 - [2025-08-31T11:07:42.932064+00:00] Ingesta PDF en /Users/anabelen.ballesteros/Desktop/Master IA&Cloud&DevOps/LLM/llm-tenerife-assistant/data/TENERIFE.pdf
2025-08-31 13:07:43.213 | INFO     | __main__:build_or_refresh_index:61 - [2025-08-31T11:07:43.213739+00:00] Insertando 26 chunks en Chroma…
2025-08-31 13:07:43.365 | INFO     | __main__:<module>:66 - [2025-08-31T11:07:43.365087+00:00] Colección lista con 52 vectores.


In [38]:
def search(query: str, k: int = 5):
    """Consulta semántica al índice y retorna documentos, metadatos y distancias."""
    res = collection.query(query_texts=[query], n_results=k)
    docs = res["documents"][0]
    metas = res["metadatas"][0]
    dists = res["distances"][0]
    return docs, metas, dists

def retrieve_context(query: str, k: int = 5) -> tuple[str, str]:
    """
    Recupera k fragmentos y produce:
    - context_text: concatenación enumerada
    - citation_tail: sufijo con 'TENERIFE.pdf · p. X, Y' para forzar citación
    """
    docs, metas, _ = search(query, k=k)
    lines = []
    pages = []
    for i, (d, m) in enumerate(zip(docs, metas), start=1):
        page = m.get("page", "?")
        pages.append(page)
        lines.append(f"[{i}] (p. {page}) {d.strip()}")
    uniq_pages = sorted(set(pages))
    citation_tail = "[TENERIFE.pdf · p. " + ", ".join(str(p) for p in uniq_pages) + "]"
    return "\n".join(lines), citation_tail

## Generación RAG con citaciones

La respuesta debe:
- Atenerse al contexto recuperado.
- Finalizar con la citación al PDF en formato `[TENERIFE.pdf · p. …]`.

In [39]:
def ask(question: str, k: int = 5) -> str:
    """
    Generación condicionada al contexto con citación obligatoria.
    """
    context_text, citation_tail = retrieve_context(question, k=k)

    system_msg = {
        "role": "system",
        "content": (
            "Eres un asistente turístico de Tenerife. Responde SOLO con el contexto proporcionado "
            "y termina SIEMPRE con la citación exacta al PDF en formato [TENERIFE.pdf · p. ...]."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            f"Pregunta: {question}\n\n"
            f"Contexto (fragmentos recuperados):\n{context_text}\n\n"
            "Instrucciones:\n"
            "- No inventes datos fuera del contexto.\n"
            "- Si el contexto no es suficiente, dilo y sugiere consultar más información del PDF.\n"
            "- Termina SIEMPRE con la citación: " + citation_tail
        )
    }

    resp = client.chat.completions.create(
        model=MODEL,
        messages=[system_msg, user_msg],
        temperature=TEMPERATURE,
        top_p=TOP_P,
        max_tokens=MAX_TOKENS,
    )
    return resp.choices[0].message.content.strip()

## Diálogo multiturno con memoria corta

In [40]:
from collections import deque

class TenerifeChat:
    """Gestor de conversación con historial acotado y RAG por turno."""
    def __init__(self, max_turns: int = 8, k: int = 5):
        self.history = deque(maxlen=max_turns * 2)  # pares user/assistant
        self.k = k

    def run(self, user_text: str) -> str:
        context_text, citation_tail = retrieve_context(user_text, k=self.k)

        system_msg = {
            "role": "system",
            "content": (
                "Eres un asistente turístico de Tenerife. Responde en español, "
                "usa el contexto proporcionado y termina con citación al PDF."
            )
        }
        messages = [system_msg] + list(self.history) + [
            {
                "role": "user",
                "content": (
                    f"Pregunta: {user_text}\n\n"
                    f"Contexto:\n{context_text}\n\n"
                    "Termina con la citación: " + citation_tail
                )
            }
        ]

        resp = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            max_tokens=MAX_TOKENS,
        )
        answer = resp.choices[0].message.content.strip()

        self.history.append({"role": "user", "content": user_text})
        self.history.append({"role": "assistant", "content": answer})
        return answer

## Tools: `get_weather` (simulado) y `get_sunlight` (Astral)

- `get_weather`: valida fecha/lugar y devuelve un pronóstico *simulado*.
- `get_sunlight`: calcula amanecer/atardecer reales para lugares de Tenerife.

In [41]:
# get_weather (simulado)
VALID_PLACES = {
    "santa cruz": "Santa Cruz de Tenerife",
    "la laguna": "San Cristóbal de La Laguna",
    "costa adeje": "Costa Adeje",
    "la orotava": "La Orotava",
}

def norm_place(text: str) -> str | None:
    key = re.sub(r"\s+", " ", text or "").strip().lower()
    return VALID_PLACES.get(key)

def get_weather(fecha: str, lugar: str) -> dict:
    """
    Pronóstico simulado: devuelve un rango de temperatura y un resumen fijo.
    Valida la fecha (ISO) y el lugar (catálogo interno).
    """
    _ = date.fromisoformat(fecha)  # raises ValueError si no es ISO
    pretty = norm_place(lugar)
    if not pretty:
        raise ValueError("Lugar no reconocido. Usa: Santa Cruz, La Laguna, Costa Adeje, La Orotava.")
    # simulación simple por día del mes
    dd = int(fecha.split("-")[2])
    tmin = 18 + (dd % 4)
    tmax = 24 + (dd % 5)
    return {
        "fecha": fecha,
        "lugar": pretty,
        "resumen": "Soleado con intervalos nubosos · (pronóstico simulado)",
        "temperatura_min": tmin,
        "temperatura_max": tmax,
        "unidad": "°C",
    }

# get_sunlight (Astral) 
from astral import LocationInfo
from astral.sun import sun

COORDS = {
    "La Laguna": (28.4893, -16.3159),
    "Santa Cruz de Tenerife": (28.4636, -16.2518),
    "Costa Adeje": (28.1210, -16.7260),
    "La Orotava": (28.3908, -16.5230),
}

def get_sunlight(fecha: str, lugar: str) -> dict:
    """
    Amanecer/atardecer (hora local) con coordenadas predefinidas por lugar.
    """
    _ = date.fromisoformat(fecha)
    pretty = norm_place(lugar) or lugar
    name = pretty if pretty in COORDS else "La Laguna"
    lat, lon = COORDS.get(name, COORDS["La Laguna"])
    loc = LocationInfo(name=name, region="ES", timezone="Atlantic/Canary", latitude=lat, longitude=lon)
    s = sun(loc.observer, date=date.fromisoformat(fecha), tzinfo=loc.timezone)
    return {
        "lugar": name,
        "fecha": fecha,
        "amanecer": s["sunrise"].strftime("%H:%M"),
        "atardecer": s["sunset"].strftime("%H:%M"),
        "tz": str(loc.timezone),
    }

## Esquemas de tools (Function Calling) y `tool_router` con Pydantic (BONUS)

In [42]:
from pydantic import BaseModel, Field, ValidationError

# Schema Pydantic para validación de get_weather
class WeatherArgs(BaseModel):
    fecha: str = Field(..., description="Fecha en formato YYYY-MM-DD")
    lugar: str = Field(..., description="Lugar en Tenerife: Santa Cruz, La Laguna, Costa Adeje, La Orotava")

    def as_tuple(self) -> tuple[str, str]:
        # Comprobar formato
        _ = date.fromisoformat(self.fecha)
        return self.fecha, self.lugar

def get_weather_validated(payload: dict) -> dict:
    """Valida entrada con Pydantic y delega en `get_weather`."""
    logger.info(f"[tool:get_weather] in={payload}")
    try:
        args = WeatherArgs(**payload)
        out = get_weather(*args.as_tuple())
        logger.info(f"[tool:get_weather] out={out}")
        return out
    except ValidationError as e:
        logger.error(f"[tool:get_weather] validation_error={e.errors()}")
        return {"error": "Parámetros inválidos. Usa fecha YYYY-MM-DD y un lugar válido de Tenerife."}
    except Exception as e:
        logger.exception(f"[tool:get_weather] unexpected_error={e}")
        return {"error": "No fue posible obtener el pronóstico en este momento."}

# Especificación de tools para el LLM 
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Pronóstico simulado para una fecha y lugar de Tenerife",
            "parameters": {
                "type": "object",
                "properties": {
                    "fecha": {"type": "string", "description": "YYYY-MM-DD"},
                    "lugar": {"type": "string", "description": "Santa Cruz, La Laguna, Costa Adeje, La Orotava"},
                },
                "required": ["fecha", "lugar"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_sunlight",
            "description": "Horarios de amanecer y atardecer para lugares de Tenerife",
            "parameters": {
                "type": "object",
                "properties": {
                    "fecha": {"type": "string", "description": "YYYY-MM-DD"},
                    "lugar": {"type": "string", "description": "Santa Cruz, La Laguna, Costa Adeje, La Orotava"},
                },
                "required": ["fecha", "lugar"],
            },
        },
    },
]

# Router de tools (usa Pydantic para weather) 
def tool_router(name: str, arguments: dict) -> dict:
    """
    Ejecuta la tool solicitada y devuelve un dict serializable.
    """
    try:
        if name == "get_weather":
            return get_weather_validated(arguments)
        if name == "get_sunlight":
            return get_sunlight(**arguments)
        return {"error": f"Tool desconocida: {name}"}
    except Exception as e:
        logger.exception(f"[tool_router] name={name} args={arguments} err={e}")
        return {"error": "Error interno al ejecutar la herramienta."}

## Orquestación con Function Calling

1. El LLM decide si llamar a una tool (`auto`).
2. Si hay `tool_calls`, se ejecuta Python y se retornan mensajes de tool.
3. Segunda llamada: el LLM redacta la respuesta final.  
Siempre cerramos con citación al PDF.

In [43]:
SYSTEM_TOOLS = {
    "role": "system",
    "content": (
        "Eres un asistente turístico que puede decidir si llamar herramientas. "
        "Si la pregunta es sobre el tiempo en una fecha/lugar de Tenerife, usa get_weather. "
        "Si preguntan por amanecer/atardecer, usa get_sunlight. "
        "Para información turística general, usa el contexto RAG."
    ),
}

def chat_with_tools(user_msg: str, k: int = 5) -> str:
    """Orquesta RAG + tools con dos llamadas al modelo."""
    context_text, citation_tail = retrieve_context(user_msg, k=k)

    messages = [
        SYSTEM_TOOLS,
        {"role": "user", "content": f"{user_msg}\n\nContexto:\n{context_text}"},
    ]

    # 1. Decisión de tools
    first = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        max_tokens=MAX_TOKENS,
        tools=TOOLS,
        tool_choice="auto",
    )

    tool_calls = first.choices[0].message.tool_calls or []
    if not tool_calls:
        # Respuesta directa + citación
        final_msgs = messages + [
            {"role": "assistant", "content": first.choices[0].message.content or ""}
        ]
        final = client.chat.completions.create(
            model=MODEL,
            messages=final_msgs + [
                {"role": "user", "content": "Termina con la citación: " + citation_tail}
            ],
            temperature=TEMPERATURE,
            top_p=TOP_P,
            max_tokens=MAX_TOKENS,
        )
        return final.choices[0].message.content.strip()

    # 2. Ejecución de tools
    messages.append(first.choices[0].message)
    for call in tool_calls:
        name = call.function.name
        arguments = json.loads(call.function.arguments or "{}")
        tool_result = tool_router(name, arguments)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": call.id,
                "name": name,
                "content": json.dumps(tool_result, ensure_ascii=False),
            }
        )

    # 3. Respuesta final + citación
    final = client.chat.completions.create(
        model=MODEL,
        messages=messages + [
            {"role": "user", "content": "Termina con la citación: " + citation_tail}
        ],
        temperature=TEMPERATURE,
        top_p=TOP_P,
        max_tokens=MAX_TOKENS,
    )
    return final.choices[0].message.content.strip()

## Pruebas automáticas y métricas (BONUS)

Incluye:
- **≥ 3** preguntas que deben invocar `get_weather`.
- Preguntas de RAG “puro”.
- Pregunta de `get_sunlight`.
- Métricas:
  - % con citación,
  - nº de llamadas (heurístico) a cada tool,
  - latencia media.

In [44]:
import time
from collections import defaultdict

tests = [
    # RAG
    "Itinerario sencillo por Santa Cruz para medio día.",
    "¿Qué recomiendan ver en La Orotava?",

    # get_weather (>= 3)
    "¿Cómo estará el tiempo el 2025-09-12 en Costa Adeje?",
    "Dime el pronóstico para 2025-09-13 en Santa Cruz de Tenerife",
    "Pronóstico para 2025-09-14 en La Laguna, por favor",

    # get_sunlight
    "¿A qué hora amanece y atardece el 2025-09-12 en La Laguna?",
]

def infer_tools_from_text(text: str) -> set[str]:
    used = set()
    if re.search(r"\b(Pronóstico|temperatura|min|max)\b", text, re.IGNORECASE):
        used.add("get_weather")
    if re.search(r"\b(Amanecer|Atardecer|Salida del sol|Puesta del sol)\b", text, re.IGNORECASE):
        used.add("get_sunlight")
    return used

results, tool_counts, latencies = [], defaultdict(int), []

for q in tests:
    t0 = time.perf_counter()
    out = chat_with_tools(q)
    dt = time.perf_counter() - t0
    latencies.append(dt)

    for tool in infer_tools_from_text(out):
        tool_counts[tool] += 1

    has_citation = bool(re.search(r"\[TENERIFE\.pdf.*p\.\s*\d+", out))
    results.append({
        "question": q,
        "has_citation": has_citation,
        "latency_s": round(dt, 2),
        "respuesta_completa": out  
    })

# Métricas agregadas
n = len(results)
pct_citation = 100 * sum(r["has_citation"] for r in results) / n if n else 0.0
avg_latency = sum(latencies)/n if n else 0.0

print("--- RESULTADOS COMPLETOS ---")
for r in results:
    print(f"- Q: {r['question']}\n  • citación={r['has_citation']} • t={r['latency_s']}s")
    print(f"  • Respuesta:\n{r['respuesta_completa']}\n")
    print("-"*120)

print("--- MÉTRICAS ---")
print({
    "n_tests": n,
    "%_respuestas_con_citación": round(pct_citation, 1),
    "latencia_media_s": round(avg_latency, 2),
    "calls_get_weather(heurístico)": tool_counts["get_weather"],
    "calls_get_sunlight(heurístico)": tool_counts["get_sunlight"],
})

2025-08-31 13:08:11.272 | INFO     | __main__:get_weather_validated:15 - [tool:get_weather] in={'fecha': '2025-09-12', 'lugar': 'Costa Adeje'}
2025-08-31 13:08:11.273 | INFO     | __main__:get_weather_validated:19 - [tool:get_weather] out={'fecha': '2025-09-12', 'lugar': 'Costa Adeje', 'resumen': 'Soleado con intervalos nubosos · (pronóstico simulado)', 'temperatura_min': 18, 'temperatura_max': 26, 'unidad': '°C'}
2025-08-31 13:08:14.632 | INFO     | __main__:get_weather_validated:15 - [tool:get_weather] in={'fecha': '2025-09-13', 'lugar': 'Santa Cruz'}
2025-08-31 13:08:14.633 | INFO     | __main__:get_weather_validated:19 - [tool:get_weather] out={'fecha': '2025-09-13', 'lugar': 'Santa Cruz de Tenerife', 'resumen': 'Soleado con intervalos nubosos · (pronóstico simulado)', 'temperatura_min': 19, 'temperatura_max': 27, 'unidad': '°C'}
2025-08-31 13:08:18.138 | INFO     | __main__:get_weather_validated:15 - [tool:get_weather] in={'fecha': '2025-09-14', 'lugar': 'La Laguna'}
2025-08-31 13

In [45]:
# Crear instancia de chat multiturno
chat = TenerifeChat()

# Turno 1
q1 = "¿Qué puedo visitar en La Orotava?"
r1 = chat.run(q1)
print("Turno 1:", q1)
print("Respuesta:\n", r1)

# Turno 2
q2 = "¿Y qué restaurantes recomiendan cerca?"
r2 = chat.run(q2)
print("\nTurno 2:", q2)
print("Respuesta:\n", r2)

Turno 1: ¿Qué puedo visitar en La Orotava?
Respuesta:
 En La Orotava, hay varios lugares interesantes que puedes visitar. Te recomiendo comenzar en la conocida Plaza de Anita, donde podrás disfrutar del Liceo de Taoro y los Jardines Victoria. Luego, puedes recorrer la Calle de la Carrera, donde se encuentran el ayuntamiento de La Orotava, la Iglesia de La Concepción, la Casa Lercaro y la Casa de los Balcones. Al final de esta calle, no te pierdas el molino de Chano, donde puedes comprar gofio, un producto típico de la zona.

Además, es importante mencionar que el casco histórico de La Orotava es un Conjunto Histórico Artístico Nacional y Patrimonio Cultural Europeo. También es el municipio más alto de España, con un gran desnivel que va desde el mar hasta el pico del Teide. Si tienes la oportunidad de visitar durante las fiestas de La Orotava, podrás ver la famosa alfombra gigante de arenas y tierras del Teide, que tiene el récord Guinness del mayor tapiz de tierra del mundo.

Por últi